In [ ]:
import numpy as np
import open3d as o3d
import trimesh
import pyvista as pv
import plotly.graph_objects as go
from pathlib import Path

class PointCloudMeshVisualizer:
    def __init__(self):
        self.point_cloud = None
        self.mesh = None

    def load_point_cloud(self, points):
        """Create point cloud from numpy array of points"""
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.estimate_normals()
        self.point_cloud = pcd
        return pcd

    def load_mesh(self, vertices, faces):
        """Create mesh from vertices and faces"""
        mesh = o3d.geometry.TriangleMesh()
        mesh.vertices = o3d.utility.Vector3dVector(vertices)
        mesh.triangles = o3d.utility.Vector3iVector(faces)
        mesh.compute_vertex_normals()
        self.mesh = mesh
        return mesh

    def visualize_point_cloud_plotly(self):
        """Visualize point cloud using Plotly"""
        if self.point_cloud is None:
            raise ValueError("Point cloud not loaded")

        points = np.asarray(self.point_cloud.points)

        fig = go.Figure(data=[
            go.Scatter3d(
                x=points[:, 0],
                y=points[:, 1],
                z=points[:, 2],
                mode='markers',
                marker=dict(
                    size=2,
                    color=points[:, 2],
                    colorscale='Viridis',
                )
            )
        ])

        fig.update_layout(
            scene=dict(
                aspectmode='data'
            ),
            title="Point Cloud Visualization"
        )
        return fig

    def visualize_mesh_pyvista(self):
        """Visualize mesh using PyVista"""
        if self.mesh is None:
            raise ValueError("Mesh not loaded")

        # Convert to PyVista mesh
        vertices = np.asarray(self.mesh.vertices)
        faces = np.asarray(self.mesh.triangles)

        mesh = pv.PolyData(vertices, faces)
        plotter = pv.Plotter()
        plotter.add_mesh(mesh, show_edges=True)
        return plotter

    def export_mesh(self, filepath, format='obj'):
        """Export mesh to file"""
        if self.mesh is None:
            raise ValueError("Mesh not loaded")

        supported_formats = ['obj', 'ply', 'stl']
        if format not in supported_formats:
            raise ValueError(f"Unsupported format. Use one of {supported_formats}")

        filepath = Path(filepath)
        if format == 'obj':
            o3d.io.write_triangle_mesh(str(filepath), self.mesh)
        elif format == 'ply':
            o3d.io.write_triangle_mesh(str(filepath), self.mesh)
        elif format == 'stl':
            # Convert to trimesh for STL export
            vertices = np.asarray(self.mesh.vertices)
            faces = np.asarray(self.mesh.triangles)
            mesh_trimesh = trimesh.Trimesh(vertices=vertices, faces=faces)
            mesh_trimesh.export(filepath)

def main():
    # Example usage
    visualizer = PointCloudMeshVisualizer()

    # Create sample point cloud
    points = np.random.rand(1000, 3)
    pcd = visualizer.load_point_cloud(points)

    # Create sample mesh (cube)
    vertices = np.array([
        [0, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [1, 1, 1],
        [0, 1, 1]
    ])
    faces = np.array([
        [0, 1, 2],
        [0, 2, 3],
        [4, 5, 6],
        [4, 6, 7],
        [0, 1, 5],
        [0, 5, 4],
        [1, 2, 6],
        [1, 6, 5],
        [2, 3, 7],
        [2, 7, 6],
        [3, 0, 4],
        [3, 4, 7]
    ])
    mesh = visualizer.load_mesh(vertices, faces)

    # Visualize
    fig = visualizer.visualize_point_cloud_plotly()
    fig.show()

    plotter = visualizer.visualize_mesh_pyvista()
    plotter.show()

if __name__ == "__main__":
    main()